In [ ]:
# Setup with LiteLLM
import os
from anthropic import Anthropic
from dotenv import load_dotenv
import json
import sqlite3
from datetime import datetime
from typing import Dict, Any

# Load environment variables
load_dotenv()

# Initialize Anthropic client (points to LiteLLM)
client = Anthropic(
    base_url=os.environ.get("ANTHROPIC_BASE_URL"),
    api_key=os.environ.get("ANTHROPIC_AUTH_TOKEN")
)

MODEL = os.environ.get("ANTHROPIC_MODEL", "claude-sonnet-4-20250514")

print("✅ Plan & Execute Agent - Using Claude via LiteLLM")
print(f"🔗 Endpoint: {os.environ.get('ANTHROPIC_BASE_URL')}")
print(f"🤖 Model: {MODEL}")

In [ ]:
# Initialize Inventory Database
from db_helper import init_inventory_database

print(init_inventory_database())

## Understanding the Planning Pattern

### 🎯 THE PLAN-AND-EXECUTE PATTERN

This is different from ReAct!

#### ReAct Pattern (Notebook 2):
```
┌────────────────────────────────┐
│ Think → Act → Observe → Think  │ ← Interleaved
│   ↓      ↓      ↓       ↓      │
│ Think → Act → Observe → Answer │
└────────────────────────────────┘
```

#### Planning Pattern (This notebook):
```
┌────────────────────────────────┐
│ Phase 1: PLANNING              │
│   Think                        │
│   Think                        │ ← All planning first
│   Think                        │
│   → Create Complete Plan       │
│                                │
│ Phase 2: EXECUTION             │
│   Act                          │
│   Act                          │ ← Then execute
│   Act                          │
│   → Return Results             │
└────────────────────────────────┘
```

#### When to use Planning Agent:

✅ Complex analytical tasks

✅ Need audit trail / transparency

✅ Want to review plan before execution

✅ Tasks with clear sub-steps

✅ Cost optimization (plan once, execute efficiently)

#### When to use ReAct:

✅ Exploratory tasks

✅ Uncertain number of steps

✅ Need adaptive decision-making

✅ Real-time responsiveness


In [ ]:
# Phase 1 - Planning Function
def create_plan(task_description: str) -> str:
    """
    Phase 1: Create a detailed plan WITHOUT executing anything
    
    The agent thinks deeply about the task and creates a step-by-step plan.
    No tools are used in this phase - pure reasoning.
    
    Args:
        task_description: What the user wants to accomplish
        
    Returns:
        A detailed plan as a string
    """
    
    planning_prompt = f"""You are a planning assistant for an inventory management system.

                        Given a task, create a detailed step-by-step plan to accomplish it.
                        
                        Available tools you can use in the execution phase:
                        - check_stock(product_id): Check current stock level for a product
                        - search_inventory(category, low_stock_only): Search inventory
                        - get_sales_trend(product_id): Get sales trend and forecast
                        - create_purchase_order(product_id, quantity, reason): Create purchase order
                        
                        Your plan should:
                        1. Break the task into clear, sequential steps
                        2. Specify which tool to use for each step
                        3. Explain the reasoning for each step
                        4. Be thorough and complete
                        
                        Output ONLY the plan as a numbered list. Be specific and actionable.
                        
                        Task: {task_description}"""
    
    print("🤔 PHASE 1: PLANNING")
    print("="*80)
    print("Creating comprehensive plan...\n")
    
    # Create plan without tools (pure reasoning)
    response = client.messages.create(
        model=MODEL,
        max_tokens=2048,
        messages=[
            {
                "role": "user",
                "content": planning_prompt
            }
        ]
    )
    
    # Extract plan
    plan = ""
    for content_block in response.content:
        if hasattr(content_block, "text"):
            plan += content_block.text
    
    print("📋 PLAN CREATED:")
    print("-"*80)
    print(plan)
    print("-"*80)
    
    return plan

In [ ]:
# Phase 2 - Execution Function
from inventory_tools import INVENTORY_TOOLS, check_stock, search_inventory, get_sales_trend, create_purchase_order


def execute_plan(task_description: str, plan: str, max_iterations: int = 15):
    """
    Phase 2: Execute the plan step by step using tools
    
    The agent follows the plan and uses tools to accomplish each step.
    This is similar to ReAct but guided by the pre-made plan.
    
    Args:
        task_description: Original task
        plan: The plan created in phase 1
        max_iterations: Safety limit
        
    Returns:
        Final results
    """
    
    print("⚙️ PHASE 2: EXECUTION")
    print("="*80)
    print("Executing plan step by step...")
    
    messages = [
        {
            "role": "user",
            "content": f"""Execute this plan step by step:
                        PLAN:
                        {plan}
                        
                        ORIGINAL TASK:
                        {task_description}
                        
                        Follow the plan carefully. Use the available tools to complete each step.
                        Show your progress as you work through the plan."""
        }
    ]
    
    system_prompt = """You are executing a pre-made plan for inventory management.
                        Follow the plan step by step. Use tools as specified in the plan.
                        After each tool call, explain what you learned and what the next step is.
                        Be thorough and complete all steps in the plan."""
    
    for iteration in range(max_iterations):
        print(f"📍 Execution Step {iteration + 1}/{max_iterations}")
        print("-"*60)
        
        # Agent executes next step
        response = client.messages.create(
            model=MODEL,
            max_tokens=2048,
            system=system_prompt,
            messages=messages,
            tools=INVENTORY_TOOLS
        )
        
        # Show agent's progress
        for content_block in response.content:
            if hasattr(content_block, "text") and content_block.text:
                print(f"💭 Agent: {content_block.text}")
        
        # Check if done
        if response.stop_reason != "tool_use":
            print("="*80)
            print("✅ EXECUTION COMPLETE")
            print("="*80)
            
            # Extract final result
            final_result = ""
            for content_block in response.content:
                if hasattr(content_block, "text"):
                    final_result += content_block.text
            
            print(f"📊 FINAL RESULTS:\n{final_result}")
            return final_result
        
        # Add assistant response to history
        messages.append({
            "role": "assistant",
            "content": response.content
        })
        
        # Execute tools
        tool_results = []
        for content_block in response.content:
            if content_block.type == "tool_use":
                tool_name = content_block.name
                tool_input = content_block.input
                
                print(f"🔧 Executing: {tool_name}")
                print(f"   Input: {json.dumps(tool_input, indent=6)}")
                
                # Execute function
                if tool_name == "check_stock":
                    result = check_stock(**tool_input)
                elif tool_name == "search_inventory":
                    result = search_inventory(**tool_input)
                elif tool_name == "get_sales_trend":
                    result = get_sales_trend(**tool_input)
                elif tool_name == "create_purchase_order":
                    result = create_purchase_order(**tool_input)
                else:
                    result = {"error": f"Unknown function: {tool_name}"}
                
                print(f"   📊 Result: {json.dumps(result, indent=6)}")
                
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": content_block.id,
                    "content": json.dumps(result)
                })
        
        # Add tool results
        messages.append({
            "role": "user",
            "content": tool_results
        })
    
    print("⚠️ Max iterations reached")
    return "Execution incomplete - reached maximum iterations"

In [ ]:
# Complete Planning Agent
def planning_agent(task_description: str, max_iterations: int = 15):
    """
    Complete Planning Agent: Plan first, then execute
    
    This combines both phases:
    1. Create a comprehensive plan
    2. Execute the plan step by step
    
    Args:
        task_description: What to accomplish
        max_iterations: Max execution steps
        
    Returns:
        Final results
    """
    
    print("="*80)
    print(f"🎯 TASK: {task_description}")
    print("="*80)
    print()
    
    # Phase 1: Create the plan
    plan = create_plan(task_description)
    
    # Phase 2: Execute the plan
    result = execute_plan(task_description, plan, max_iterations)
    
    return result

In [ ]:
# Test 1 - Simple Inventory Check
print("TEST 1: Simple Inventory Check")

planning_agent("Show me all products that are currently low on stock")

In [ ]:
# Test 2 - Sales Analysis
print("TEST 2: Sales Trend Analysis")

planning_agent("Analyze the sales trend for the USB-C Cable (PROD002) and recommend whether we should reorder")

In [ ]:
# Test 3 - Complex Multi-Step Task
print("TEST 3: Complete Inventory Audit and Restocking")

planning_agent("""
                Perform a comprehensive inventory audit:
                1. Find all products that are low on stock
                2. For each low-stock product, check its sales trend
                3. Create purchase orders for products that will run out in the next 7 days
                4. Provide a summary report of all actions taken
                """)

In [ ]:
# Test 4 - Category Analysis
print("TEST 4: Category-Based Inventory Management")

planning_agent("""
                Analyze the Electronics category:
                1. Find all electronics products
                2. Check which ones are low on stock
                3. For low stock items, analyze their sales trends
                4. Recommend reorder quantities based on average daily sales
                5. Create purchase orders for critical items
                """)

## Understanding the Two Phases

### 🎓 WHAT YOU JUST SAW: Plan-and-Execute in Action

Let's analyze what happened in Test 3:

#### PHASE 1: PLANNING (1 API call)

---

🤔 **Agent thinks deeply:**

"To audit inventory, I need to:
1. Search for low stock items
2. Check sales trend for each
3. Calculate days until stockout
4. Create purchase orders if needed
5. Summarize findings"

**Result:** Complete plan with 5 steps

#### PHASE 2: EXECUTION (5-10 API calls)

---

**Step 1:** 🔧 `search_inventory(low_stock_only=True)`

**Step 2:** 🔧 `get_sales_trend(PROD002)`

**Step 3:** 🔧 `get_sales_trend(PROD005)`

**Step 4:** 🔧 `create_purchase_order(PROD002, ...)`

**Step 5:** 💬 Provide summary report

#### KEY INSIGHTS:

---

✅ Planning happens ONCE upfront

✅ Execution follows the plan systematically

✅ More predictable than ReAct

✅ Easier to audit and explain

✅ Can review/modify plan before execution

✅ Better for tasks with clear structure